In [1]:
from google.colab import drive
drive.mount( '/content/drive')
%cd /content/drive/Shareddrives/fine_tuning/AI_LLM/

Mounted at /content/drive
/content/drive/Shareddrives/fine_tuning/AI_LLM


In [2]:
!pip install transformers datasets peft evaluate faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install llama-index llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-readers-web python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19

In [4]:
!pip install trl tensorboard bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.5 MB/s eta 0:00:00


In [107]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader

from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    Trainer,
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    TrainerCallback,
    TrainerState,
    TrainerControl
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import evaluate
import torch
import numpy as np

In [108]:

model_checkpoint = 'distilbert-base-uncased'

# Vérifier que le modèle est correctement chargé
print(model_checkpoint)

distilbert-base-uncased


In [109]:

# Charger les données CSV
data_path = "data.csv"  # Remplacez par le chemin de votre fichier CSV
dataset = load_dataset("csv", data_files=data_path, encoding="latin-1")

# Renommer les colonnes
dataset = dataset.rename_column(original_column_name=dataset['train'].column_names[0], new_column_name="label")
dataset = dataset.rename_column(original_column_name=dataset['train'].column_names[1], new_column_name="text")

# Diviser le dataset en train et test (80% train, 20% test)
dataset_split = dataset['train'].train_test_split(test_size=0.2)
# Diviser l'ensemble d'entraînement en train (70%) et validation (10%)
dataset = dataset_split['train'].train_test_split(test_size=0.125)  # 10% du total


# Afficher les résultats
print(dataset)



DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 3391
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 485
    })
})


In [18]:
train_size = len(dataset['train'])
test_size = len(dataset['test'])

N = 100

print(f"Taille de l'ensemble d'entraînement : {train_size}")
print(f"Taille de l'ensemble de test : {test_size}")

Taille de l'ensemble d'entraînement : 3391
Taille de l'ensemble de test : 485


In [19]:
# Générer des indices aléatoires pour un sous-échantillon
rand_idx_train = np.random.randint(train_size, size=N).tolist()
rand_idx_test = np.random.randint(test_size, size=N).tolist()

# Extraire les données d'entraînement et de test
x_train = [dataset['train'][i]['text'] for i in rand_idx_train]
y_train = [dataset['train'][i]['label'] for i in rand_idx_train]

x_test = [dataset['test'][i]['text'] for i in rand_idx_test]
y_test = [dataset['test'][i]['label'] for i in rand_idx_test]

# create new dataset
dataset = DatasetDict({
    'train':Dataset.from_dict({'label':y_train,'text':x_train}),
    'validation':Dataset.from_dict({'label':y_test,'text':x_test})
    })



# Afficher le dataset résultant
print(dataset)




DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 100
    })
})


In [102]:

# define label maps
label2id = {0: "Negative", 1: "Positive", 2: "Neutral"}
id2label = {"Negative": 0, "Positive": 1, "Neutral": 2}

# Generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    label2id=label2id,
    id2label=id2label,
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Process Data

In [103]:
# Create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [104]:

# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]


    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding = True,
        truncation=True,
        max_length=512
    )

    # Convert labels to integers
    tokenized_inputs['labels'] = [id2label[label] for label in examples['label']]

    return tokenized_inputs



In [106]:

# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

KeyError: 'neutral'

In [77]:

# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='left', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

## evaluation

In [78]:

# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [79]:

# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

## Apply untrained model to text

In [80]:
# define list of examples
text_list = ["You are so ugly.", "I love you", "Not a big fan, don't recommed.", "Really better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + label2id[predictions.tolist()])

Untrained model predictions:
----------------------------
You are so ugly. - Negative
I love you - Neutral
Not a big fan, don't recommed. - Neutral
Really better than the first one. - Negative
This is not worth watching even once. - Neutral
This one is a pass. - Neutral


## Train model

In [81]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

peft_config

LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=4, target_modules={'q_lin'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [90]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# hyperparameters
lr = 1e-3
batch_size = 16
num_epochs = 10


trainable params: 629,763 || all params: 67,585,542 || trainable%: 0.9318


In [91]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [52]:
!pip install wandb

In [92]:

# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

<ipython-input-92-e8e4c1055773>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

## Generate prediction

In [113]:


print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt")

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    # Get predicted label ID
    predicted_label_id = predictions.tolist()[0]

    # Check if the predicted label ID is in id2label
    if predicted_label_id in id2label.values():  # Use .values() to check for values
        predicted_label = list(id2label.keys())[list(id2label.values()).index(predicted_label_id)]
        print(text + " - " + predicted_label)
    else:
        print(f"Warning: Prediction {predicted_label_id} is not in id2label. Text: {text}")



Trained model predictions:
--------------------------
You are so ugly. - Neutral
I love you - Neutral
Not a big fan, don't recommed. - Neutral
Really better than the first one. - Neutral
This is not worth watching even once. - Neutral
This one is a pass. - Neutral
